In [1]:
# import relevant libraries

import pandas as pd
import requests
import time
import os
import yfinance as yf

## Extract

In [2]:
# API configuration

# function that fetchs stock data for a given symbol
def fetch_stock_info(symbol):
    stock = yf.Ticker(symbol)
    # Fetch historical market data for 100 days
    df = stock.history(period="3mo")
    return df

In [3]:
# test function
# Test by fetching stock data for Apple (AAPL)
data = fetch_stock_info('AAPL')
data

Open        High         Low       Close  \
Date                                                                        
2024-06-17 00:00:00-04:00  213.123307  218.696857  212.474064  216.419495   
2024-06-18 00:00:00-04:00  217.338424  218.377230  212.753735  214.042236   
2024-06-20 00:00:00-04:00  213.682663  213.992317  208.608549  209.437576   
2024-06-21 00:00:00-04:00  210.146763  211.645029  206.870557  207.250122   
2024-06-24 00:00:00-04:00  207.479840  212.454078  206.351142  207.899353   
...                               ...         ...         ...         ...   
2024-09-10 00:00:00-04:00  218.919998  221.479996  216.729996  220.110001   
2024-09-11 00:00:00-04:00  221.460007  223.089996  217.889999  222.660004   
2024-09-12 00:00:00-04:00  222.500000  223.550003  219.820007  222.770004   
2024-09-13 00:00:00-04:00  223.580002  224.039993  221.910004  222.500000   
2024-09-16 00:00:00-04:00  216.740005  217.199997  213.919998  216.350006   

                              Volume  Dividends  Stock Splits  
Date                                                           
2024-06-17 00:00:00-04:00   93728300        0.0           0.0  
2024-06-18 00:00:00-04:00   79943300        0.0           0.0  
2024-06-20 00:00:00-04:00   86172500        0.0           0.0  
2024-06-21 00:00:00-04:00  246421400        0.0           0.0  
2024-06-24 00:00:00-04:00   80727000        0.0           0.0  
...                              ...        ...           ...  
2024-09-10 00:00:00-04:00   51591000        0.0           0.0  
2024-09-11 00:00:00-04:00   44587100        0.0           0.0  
2024-09-12 00:00:00-04:00   37498200        0.0           0.0  
2024-09-13 00:00:00-04:00   36722900        0.0           0.0  
2024-09-16 00:00:00-04:00   44235855        0.0           0.0  

[63 rows x 7 columns]

## Transform

In [4]:
# define function that processes and transforms the stock data

def process_stock_data(df, symbol, sp500_info):
    # Ensure the index is in datetime format
    df.index = pd.to_datetime(df.index)
    
    # Add the stock symbol to the DataFrame
    df['symbol'] = symbol
    
    # Handle missing values by forward filling
    df.fillna(method='ffill', inplace=True)
    
    # Calculate moving averages
    df['moving_avg_7'] = df['Close'].rolling(window=7).mean()
    df['moving_avg_30'] = df['Close'].rolling(window=30).mean()
    
    # Normalize the close price
    df['normalized_close'] = (df['Close'] - df['Close'].min()) / (df['Close'].max() - df['Close'].min())
    
    # Flag high volatility days
    df['volatility'] = df['High'] - df['Low']
    df['high_volatility_flag'] = df['volatility'] > df['volatility'].mean()
    
    # Detect outliers in the close price
    df['price_outlier'] = (df['Close'] > df['Close'].mean() + 3 * df['Close'].std()) | (df['Close'] < df['Close'].mean() - 3 * df['Close'].std())
    
    # Merge S&P 500 company information into the DataFrame
    company_info = sp500_info[sp500_info['Symbol'] == symbol]
    
    # Add the columns for 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location'
    if not company_info.empty:
        df['Security'] = company_info['Security'].values[0]
        df['GICS Sector'] = company_info['GICS Sector'].values[0]
        df['GICS Sub-Industry'] = company_info['GICS Sub-Industry'].values[0]
        df['Headquarters Location'] = company_info['Headquarters Location'].values[0]
    
    # Return the transformed DataFrame
    return df


## Load (to local storage)

In [5]:
# directory that saves the 100 days stock data
data_dir = r"C:\Users\adityamxr\Desktop\Projects\stock-market-analysis\data\stock_data"
os.makedirs(data_dir, exist_ok=True)

In [6]:
# read the S&P 500 company list with additional information
sp500_df = pd.read_csv(r"C:\Users\adityamxr\Desktop\Projects\stock-market-analysis\data\s&p500.csv")

# extract stock symbols
stock_symbols = sp500_df['Symbol'].tolist()

In [7]:
# fetch and save stock data for each company
for symbol in stock_symbols:
    try:
        print(f"Fetching data for {symbol}...")
        stock_data = fetch_stock_info(symbol)  # Fetch stock data using yfinance
        processed_data = process_stock_data(stock_data, symbol, sp500_df)  # Process the stock data

        # Save the processed data to CSV
        processed_data.to_csv(f'{data_dir}/{symbol}_stock_data.csv', index=True)
        print(f"Saved data for {symbol}.")

        # Respect API rate limits (if any)
        time.sleep(1)  # Short sleep to prevent overloading

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")


Fetching data for A...
Saved data for A.
Fetching data for AAL...
Saved data for AAL.
Fetching data for AAPL...
Saved data for AAPL.
Fetching data for ABBV...
Saved data for ABBV.
Fetching data for ABNB...
Saved data for ABNB.
Fetching data for ABT...
Saved data for ABT.
Fetching data for ACGL...
Saved data for ACGL.
Fetching data for ACN...
Saved data for ACN.
Fetching data for ADBE...
Saved data for ADBE.
Fetching data for ADI...
Saved data for ADI.
Fetching data for ADM...
Saved data for ADM.
Fetching data for ADP...
Saved data for ADP.
Fetching data for ADSK...
Saved data for ADSK.
Fetching data for AEE...
Saved data for AEE.
Fetching data for AEP...
Saved data for AEP.
Fetching data for AES...
Saved data for AES.
Fetching data for AFL...
Saved data for AFL.
Fetching data for AIG...
Saved data for AIG.
Fetching data for AIZ...
Saved data for AIZ.
Fetching data for AJG...
Saved data for AJG.
Fetching data for AKAM...
Saved data for AKAM.
Fetching data for ALB...
Saved data for ALB.


$BF.B: possibly delisted; no price data found  (period=3mo)


Fetching data for BF.B...
Saved data for BF.B.
Fetching data for BG...
Saved data for BG.
Fetching data for BIIB...
Saved data for BIIB.
Fetching data for BIO...
Saved data for BIO.
Fetching data for BK...
Saved data for BK.
Fetching data for BKNG...
Saved data for BKNG.
Fetching data for BKR...
Saved data for BKR.
Fetching data for BLDR...
Saved data for BLDR.
Fetching data for BLK...
Saved data for BLK.
Fetching data for BMY...
Saved data for BMY.
Fetching data for BR...
Saved data for BR.
Fetching data for BRK.B...


$BRK.B: possibly delisted; no price data found  (period=3mo) (Yahoo error = "No data found, symbol may be delisted")


Saved data for BRK.B.
Fetching data for BRO...
Saved data for BRO.
Fetching data for BSX...
Saved data for BSX.
Fetching data for BWA...
Saved data for BWA.
Fetching data for BX...
Saved data for BX.
Fetching data for BXP...
Saved data for BXP.
Fetching data for C...
Saved data for C.
Fetching data for CAG...
Saved data for CAG.
Fetching data for CAH...
Saved data for CAH.
Fetching data for CARR...
Saved data for CARR.
Fetching data for CAT...
Saved data for CAT.
Fetching data for CB...
Saved data for CB.
Fetching data for CBOE...
Saved data for CBOE.
Fetching data for CBRE...
Saved data for CBRE.
Fetching data for CCI...
Saved data for CCI.
Fetching data for CCL...
Saved data for CCL.
Fetching data for CDNS...
Saved data for CDNS.
Fetching data for CDW...
Saved data for CDW.
Fetching data for CE...
Saved data for CE.
Fetching data for CEG...
Saved data for CEG.
Fetching data for CF...
Saved data for CF.
Fetching data for CFG...
Saved data for CFG.
Fetching data for CHD...
Saved data f

Fetching data for IRM...
Saved data for IRM.
Fetching data for ISRG...
Saved data for ISRG.
Fetching data for IT...
Saved data for IT.
Fetching data for ITW...
Saved data for ITW.
Fetching data for IVZ...
Saved data for IVZ.
Fetching data for J...
Saved data for J.
Fetching data for JBHT...
Saved data for JBHT.
Fetching data for JBL...
Saved data for JBL.
Fetching data for JCI...
Saved data for JCI.
Fetching data for JKHY...
Saved data for JKHY.
Fetching data for JNJ...
Saved data for JNJ.
Fetching data for JNPR...
Saved data for JNPR.
Fetching data for JPM...
Saved data for JPM.
Fetching data for K...
Saved data for K.
Fetching data for KDP...
Saved data for KDP.
Fetching data for KEY...
Saved data for KEY.
Fetching data for KEYS...
Saved data for KEYS.
Fetching data for KHC...
Saved data for KHC.
Fetching data for KIM...
Saved data for KIM.
Fetching data for KKR...
Saved data for KKR.
Fetching data for KLAC...
Saved data for KLAC.
Fetching data for KMB...
Saved data for KMB.
Fetching

Fetching data for T...
Saved data for T.
Fetching data for TAP...
Saved data for TAP.
Fetching data for TDG...
Saved data for TDG.
Fetching data for TDY...
Saved data for TDY.
Fetching data for TECH...
Saved data for TECH.
Fetching data for TEL...
Saved data for TEL.
Fetching data for TER...
Saved data for TER.
Fetching data for TFC...
Saved data for TFC.
Fetching data for TFX...
Saved data for TFX.
Fetching data for TGT...
Saved data for TGT.
Fetching data for TJX...
Saved data for TJX.
Fetching data for TMO...
Saved data for TMO.
Fetching data for TMUS...
Saved data for TMUS.
Fetching data for TPR...
Saved data for TPR.
Fetching data for TRGP...
Saved data for TRGP.
Fetching data for TRMB...
Saved data for TRMB.
Fetching data for TROW...
Saved data for TROW.
Fetching data for TRV...
Saved data for TRV.
Fetching data for TSCO...
Saved data for TSCO.
Fetching data for TSLA...
Saved data for TSLA.
Fetching data for TSN...
Saved data for TSN.
Fetching data for TT...
Saved data for TT.
Fe